In [1]:
import joblib
import numpy as np
import pandas as pd

from planet.app import Planet, Geo

In [2]:
model = joblib.load('models/ridge_model.pkl')

In [3]:
def insert_param(frame: pd.Series):
    coord = Geo(frame['place']).coord
    new_data = pd.Series({        
        'Широта': coord['latitude'],
        'Долгота': coord['longitude'],        
    })

    frame = pd.concat([frame, new_data])    
    parametrs = Planet(coord['longitude'], coord['latitude'], frame['date']).get_parametrs()
    
    return pd.concat([frame, pd.Series(parametrs)])

def make_frame(place: str, date: str, period: int) -> pd.DataFrame:

    frame = pd.DataFrame(
        data=pd.date_range(date, periods=period, freq="D").to_list(),
        columns=['date']
    )
    frame['place'] = place
    
    return frame.apply(insert_param, axis=1)

def predict_magnitude(place: str, date: str, period: int):
    frame = make_frame(place, date, period)
    frame['magnitude'] = model.predict(frame.drop(['date', 'place'], axis=1))
    return frame[['date', 'magnitude']]

In [8]:
place = 'Barnaul, Russia'
date = "2023-02-11"  # начальная дата периода
period = 700  # длительность периода  днях

magnitude_frame = predict_magnitude(place, date, period)

In [9]:
magnitude_frame

,date,magnitude
0,2023-02-11,2.502435
1,2023-02-12,2.507417
2,2023-02-13,2.512571
3,2023-02-14,2.517716
4,2023-02-15,2.522656
...,...,...
695,2025-01-06,2.715040
696,2025-01-07,2.706692
697,2025-01-08,2.699058
698,2025-01-09,2.692593


In [11]:
magnitude_frame.describe()

,magnitude
count,700.000000
mean,2.605878
std,0.079129
min,2.464760
25%,2.542122
50%,2.594587
75%,2.670856
max,2.804459


In [12]:
magnitude_frame[magnitude_frame['magnitude'] > 2.79]

,date,magnitude
656,2024-11-28,2.790935
657,2024-11-29,2.796610
658,2024-11-30,2.798646
659,2024-12-01,2.801657
660,2024-12-02,2.803951
661,2024-12-03,2.804459
662,2024-12-04,2.803641
663,2024-12-05,2.801438
664,2024-12-06,2.797612
665,2024-12-07,2.793849
